<a href="https://colab.research.google.com/github/lavstat17/Notebook_Product_scoring/blob/main/OFCOM's_Scoring_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing libraries
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Read the Scrapped data from whole foods market website
df = pd.read_csv(r'/content/drive/MyDrive/NP_score.csv')
df

,s.no,Nameofproduct,protein_g,sodium_mg,fiber_g,sugars_g,Servingsize_g,Energy/calories,Saturated_Fat_g,Saturatedfat_DV_percentage,Sodium_mg,Sodium_DV_percentage
0,1,"Original Ancient Grain Granola, 12 oz",3,135,2,7,30,130,3.5,0.18,135,0.06
1,2,"Original Granola, 12 oz",3,0,2,6,28,150,2.0,0.10,0,0.00
2,3,"Raisin Bran Cereal, 15 oz",5,240,8,15,55,170,0.0,0.00,240,0.10
3,4,365 Everyday Value® Organic Cocoa Crisps Cerea...,2,110,1,11,40,140,0.0,0.00,110,0.05
4,5,"Cinnamon Toast Cereal, 9 oz",10,90,6,0,26,90,0.0,0.00,90,0.04
5,6,"Dark Chocolate Catalina Crunch Cereal, 9 oz",8,125,7,0,26,90,0.5,0.03,125,0.05
6,7,"Nature's Path Organic Organic Heritage Flakes,...",5,170,7,5,40,160,0.0,0.00,170,0.07
7,8,"Organic Old Fashioned Rolled Oats, 32 oz",5,0,3,1,48,180,0.5,0.03,0,0.00
8,9,"Old Country Style Muesli, 40 oz",4,0,4,5,35,140,0.0,0.00,0,0.00


In [ ]:
#convert each serving size to 100g
df_new = df[['protein_g','Saturated_Fat_g','sugars_g','Servingsize_g','sodium_mg','fiber_g','Energy/calories']].astype(int)
d = {30:3.34, 28:3.58, 55:1.83, 40:2.5, 26:3.85, 26:3.85, 40:2.5, 48:2.1, 35:2.86}
df_new['constant'] = df['Servingsize_g'].map(d)
df_new['protein_100g'] = df_new['protein_g']*df_new['constant'].astype(int)
df_new['Saturated_Fat_100g'] = df_new['Saturated_Fat_g']*df_new['constant'].astype(int)
df_new['sugars_100g'] = df_new['sugars_g']*df_new['constant']
df_new['fiber_100g'] = df_new['fiber_g']*df_new['constant']
df_new['Servingsize_100g'] = df_new['Servingsize_g']*df_new['constant']
df_new['sodium_mg'] = df_new['sodium_mg'].astype(int)
df_new['Energy_KJ'] = df_new['Energy/calories']*4.184
df_new['Energy_KJ'] = df_new['Energy_KJ'].astype(int)

In [ ]:
#negative factors dataframe
df_new1= df_new[['Energy_KJ','sodium_mg','Saturated_Fat_100g','sugars_100g']]
df_new1

,Energy_KJ,sodium_mg,Saturated_Fat_100g,sugars_100g
0,543,135,9,23.38
1,627,0,6,21.48
2,711,240,0,27.45
3,585,110,0,27.50
4,376,90,0,0.00
5,376,125,0,0.00
6,669,170,0,12.50
7,753,0,0,2.10
8,585,0,0,14.30


In [ ]:
##positive factors dataframe
df_new2= df_new[['fiber_100g','protein_100g']]
df_new2

,fiber_100g,protein_100g
0,6.68,9
1,7.16,9
2,14.64,5
3,2.50,4
4,23.10,30
5,26.95,24
6,17.50,10
7,6.30,10
8,11.44,8


In [ ]:
#Threshold values for negative factors
negative_factor = pd.read_csv(r'/content/drive/MyDrive/negative_factor.csv')
negative_factor.set_index('Points')

,Energy_KJ,Saturated_fat_g,Total_Sugar_g,Sodium_mg
Points,,,,
0,<= 335,<= 1,<=4.5,<=90
1,>335,>1,>4.5,>90
2,>670,>2,>9,>180
3,>1005,>3,>13.5,>270
4,>1340,>4,>18,>360
5,>1675,>5,>22.5,>450
6,>2010,>6,>27,>540
7,>2345,>7,>31,>630
8,>2680,>8,>36,>720


In [ ]:
#Threshold values for positive factors
positive_factor = pd.read_csv(r'/content/drive/MyDrive/positive_factors.csv')
positive_factor.set_index('Points')

,Fruit_Veg_Nuts_percentage,NSP_Fibre_g,AOAC_Fibre_g,Protein_g
Points,,,,
0,<=40,<=0.7,<=0.9,<=1.6
1,>40,>0.7,>0.9,>1.6
2,>60,>1.4,>1.9,>3.2
3,-,>2.1,>2.8,>4.8
4,-,>2.8,>3.7,>6.4
5,>80,>3.5,>4.7,>8.0


In [ ]:
## Negative factors scoring based on threshold value

bins1 = [0,335,670,1005,1340,1675,2010,2345,2680,3015,3350]
bins2 = [0,1,2,3,4,5,6,7,8,9,10]
bins3 = [0,4.5,9,13.5,18,22.5,27,31,36,40,45]
bins4 = [0,90,180,270,360,450,540,630,720,810,900]
df_new1['Energy_KJ_score'] = pd.cut(df_new1['Energy_KJ'], bins=bins1, labels=False)
df_new1['Saturated_Fat_score_'] = pd.cut(df_new1['Saturated_Fat_100g'],bins=bins2, labels=False,include_lowest=True)
df_new1['sugars_100g_score'] = pd.cut(df_new1['sugars_100g'],bins=bins3,labels=False,include_lowest=True)
df_new1['sodium_mg_score'] = pd.cut(df_new1['sodium_mg'],bins=bins4,labels=False,include_lowest=True)
df_new1

,Energy_KJ,sodium_mg,Saturated_Fat_100g,sugars_100g,Energy_KJ_score,Saturated_Fat_score_,sugars_100g_score,sodium_mg_score
0,543,135,9,23.38,1,8,5,1
1,627,0,6,21.48,1,5,4,0
2,711,240,0,27.45,2,0,6,2
3,585,110,0,27.50,1,0,6,1
4,376,90,0,0.00,1,0,0,0
5,376,125,0,0.00,1,0,0,1
6,669,170,0,12.50,1,0,2,1
7,753,0,0,2.10,2,0,0,0
8,585,0,0,14.30,1,0,3,0


In [ ]:
## Positive factors scoring based on threshold values

bin1 = [0,0.7,1.4,2.1,2.8,3.5,40]
bin2 = [0,1.6,3.2,4.8,6.4,8.0,40]
df_new2['fiber_score'] = pd.cut(df_new2['fiber_100g'], bins=bin1, labels=False,include_lowest=True)
df_new2['protein_score'] = pd.cut(df_new2['protein_100g'],bins=bin2, labels=False,include_lowest=True)
df_new2

,fiber_100g,protein_100g,fiber_score,protein_score
0,6.68,9,5,5
1,7.16,9,5,5
2,14.64,5,5,3
3,2.50,4,3,2
4,23.10,30,5,5
5,26.95,24,5,5
6,17.50,10,5,5
7,6.30,10,5,5
8,11.44,8,5,4


In [ ]:
#calculating A points
#Total 'A' points = (points for energy) + (points for saturated fat) + (points for sugars) + (points for sodium)
def A_points(row):
    return row['Energy_KJ_score']+row['sodium_mg_score']+row['Saturated_Fat_score_']+row['sugars_100g_score']

A = A_points(df_new1)
A

0    15
1    10
2    10
3     8
4     1
5     2
6     4
7     2
8     4
dtype: int64

In [ ]:
#caluclating C points
#Total 'C' points = (points for % fruit, vegetable & nut content) + (points for fibre [either NSP or AOAC]) + (points for protein)
def C_points(row):
    return row['fiber_score']+row['protein_score']

C = C_points(df_new2)
C

score = {'A':A,'C':C}
score = pd.concat(score, axis=1)
score

,A,C
0,15,10
1,10,10
2,10,8
3,8,5
4,1,10
5,2,10
6,4,10
7,2,10
8,4,9


In [ ]:
#caluclating overall score
#If a food scores less than 11 ‘A’ points then the overall score is calculated as follows:
#Total ‘A’ points (energy + saturated fat + sugars + sodium)
#Minus
#Total ‘C’ points (fruit, veg and nuts + fibre + protein)

# If a food scores 11 or more ‘A’ points but scores 5 points for fruit, vegetables and nuts then the overall score is calculated as follows:
# Total ‘A’ points (energy + saturated fat + sugars + sodium)
# Minus
# Total ‘C’ points (fruit, veg and nuts + fibre + protein)

# • If a food scores 11 or more ‘A’ points, and less than 5 points for fruit, vegetables and nuts,
# then the overall score is calculated as follows :
# Total ‘A’ points (energy + saturated fat + sugars + sodium)
# Minus
# Points for fibre + points for fruit, vegetables and nuts (not allowed to score for protein)




# Based on conditions finding overall score
def overall_score(row):
    if row['A'] < 11:
        return row['A']- row['C']

score['final_score'] = score.apply( lambda row : overall_score(row), axis = 1)
score

,A,C,final_score
0,15,10,NaN
1,10,10,0.0
2,10,8,2.0
3,8,5,3.0
4,1,10,-9.0
5,2,10,-8.0
6,4,10,-6.0
7,2,10,-8.0
8,4,9,-5.0
